In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
n_HG=64
n_WG=64
n_HD=28
n_WD=28
lr=1e-3
mb_size=32

In [4]:
mnist = input_data.read_data_sets('/home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\t10k-images-idx3-ubyte.gz
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives suc

In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample, cmap='Greys_r')

    return fig

In [6]:
#Generator Place Holder

In [7]:
z = tf.placeholder(tf.float32, [None, n_HG, n_WG,1], name='z')

In [8]:
#Discriminator Place Holder

In [9]:
x = tf.placeholder(tf.float32, [None, n_HD, n_WD, 1], name='z')

In [10]:
#Generator Weights 

In [11]:
    GW1 = tf.get_variable("GW1", [33, 33,1, 5], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    GW2 = tf.get_variable("GW2", [5, 5, 5, 1], initializer=tf.contrib.layers.xavier_initializer(seed=0))


Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#Discriminator Weights 

In [13]:
    DW1 = tf.get_variable("DW1", [4, 4, 1, 8], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    DW2 = tf.get_variable("DW2", [2, 2, 8, 16], initializer=tf.contrib.layers.xavier_initializer(seed=0))

In [14]:
def discriminator(X):
    
    
    # Retrieve the parameters from the dictionary "parameters" 
 
    ### START CODE HERE ###
    # CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X, DW1, strides=[1, 1, 1, 1], padding='VALID')
    # RELU
    A1 = tf.nn.relu(Z1)
    # MAXPOOL: window 8x8, stride 8, padding 'SAME'
    P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], strides = [1, 8, 8, 1], padding='SAME')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1, DW2, strides=[1, 1, 1, 1], padding='VALID')
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = tf.nn.max_pool(A2, ksize = [1, 4, 4, 1], strides = [1, 4, 4, 1], padding='SAME')
    # FLATTEN
    P = tf.contrib.layers.flatten(P2)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z3 = tf.contrib.layers.fully_connected(P, 1, activation_fn=None)
    D_prob=tf.nn.sigmoid(Z2)
    ### END CODE HERE ###

    return D_prob


In [15]:
def generator(Z):
    Z1=tf.nn.conv2d(z, GW1, strides=[1,1,1,1] ,padding='VALID')
    A1 = tf.nn.relu(Z1)
   #P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], strides = [1, 8, 8, 1], padding='SAME')
    
    Z2 = tf.nn.conv2d(A1, GW2, strides=[1, 1, 1, 1], padding='VALID')
    A2 = tf.nn.sigmoid(Z2)
    return A2

In [16]:
def log(x):
    return tf.log(x + 1e-8)
#loss functions

G_sample = generator(z)
D_real = discriminator(x)
D_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(log(D_real) +  log(1. - D_fake)) #mean ( logD(x)+log( 1-D( G(z) ) ) )
G_loss = -tf.reduce_mean(log(D_fake)) #mean( D ( G ( Z ) ) ) ) we maximize logD(G(Z))) instead of minizing (log(1-D(G(Z)))

Instructions for updating:
Use keras.layers.flatten instead.


In [17]:
theta_D=[DW1, DW2]
theta_G=[GW1, GW2]

In [18]:
D_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=theta_G)


In [19]:
def sample_Z(m, n, l):
    '''Uniform prior for G(Z)'''
    return np.random.uniform(-1., 1., size=[m, n, l,1])

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
for it in range(1000000):
        X_mb, _ = mnist.train.next_batch(mb_size)  
        X_mb=X_mb.reshape(mb_size, n_HD, n_WD,1)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={x: X_mb, z: sample_Z(mb_size, n_HG, n_WG)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={z: sample_Z(mb_size, n_HG, n_WG)})
        samples = sess.run(G_sample, feed_dict={z: sample_Z(16, n_HG, n_WG)})
    

        if it % 1000 == 0:

            op = open("CNN.txt","a") 
            print('Iter: {} ; D_loss: {:.4}; G_loss: {:.4} ; '
                  .format(it, D_loss_curr, G_loss_curr))
            op.write('Iter: {} ; D_loss: {:.4}; G_loss: {:.4} \n'
                    .format(it, D_loss_curr, G_loss_curr)) 
            op.close() 
            samples = sess.run(G_sample, feed_dict={z: sample_Z(16, n_HG, n_WG)})
            samples=samples.reshape(16, n_HD, n_WD)
            fig = plot(samples)
            plt.savefig('CNN/{}.png'
                        .format(str(it).zfill(3)), bbox_inches='tight')
            it += 1
            plt.close(fig)

Iter: 0 ; D_loss: 1.42; G_loss: 0.6442 ; 
Iter: 1000 ; D_loss: 1.196; G_loss: 0.726 ; 
Iter: 2000 ; D_loss: 1.129; G_loss: 0.7347 ; 
Iter: 3000 ; D_loss: 1.091; G_loss: 0.7269 ; 
Iter: 4000 ; D_loss: 1.019; G_loss: 0.7031 ; 
Iter: 5000 ; D_loss: 1.016; G_loss: 0.7194 ; 
Iter: 6000 ; D_loss: 1.011; G_loss: 0.7231 ; 
Iter: 7000 ; D_loss: 0.9592; G_loss: 0.7066 ; 
Iter: 8000 ; D_loss: 0.9493; G_loss: 0.7017 ; 


KeyboardInterrupt: 